Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [22]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq

In [23]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [24]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [ ]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state , goal_state

In [26]:
def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    distance = 0
    for num in range(1, PUZZLE_DIM**2): 
        x1, y1 = np.where(state == num)
        x2, y2 = np.where(goal == num)
        distance += abs(x1 - x2) + abs(y1 - y2)
    return int(distance)

def difference(state: np.ndarray, goal: np.ndarray) -> int:
    return np.sum(np.abs(state - goal))


def combined(state: np.ndarray, goal: np.ndarray) -> int:
    return max(manhattan_distance(state, goal), difference(state, goal))

In [31]:
def a_star(start: np.ndarray, goal: np.ndarray):
    # Priority queue (min-heap) initialized with the starting state.
    frontier = []
    start_tuple = tuple(start.flatten())
    heapq.heappush(frontier, (0, start_tuple))
    
    # Track the best cost to reach each state (number of steps).
    cost_so_far = {start_tuple: 0}
    
    # Set to keep track of visited states to avoid revisits.
    visited = set()

    # Counter to track the number of times `do_action` is called.
    action_count = 0

    while frontier:
        # Pop the state with the lowest priority (cost + heuristic).
        current_priority, current_tuple = heapq.heappop(frontier)
        current = np.array(current_tuple).reshape(PUZZLE_DIM, PUZZLE_DIM)
        
        # Check if we've reached the goal.
        if np.array_equal(current, goal):
            # Return the total number of actions taken (do_action calls).
            return action_count

        # Mark the current state as visited.
        visited.add(current_tuple)
        
        # Explore all possible moves from the current state.
        for action in available_actions(current):
            next_state = do_action(current, action)
            
            # Increment the action count for each `do_action` call.
            action_count += 1
            
            next_tuple = tuple(next_state.flatten())
            
            # Calculate the new cost for the next state (increment steps by 1).
            new_cost = cost_so_far[current_tuple] + 1
            
            # Skip if this state has already been visited.
            if next_tuple in visited:
                continue
            
            # If it's a new state or found with a lower cost, update it.
            if next_tuple not in cost_so_far or new_cost < cost_so_far[next_tuple]:
                cost_so_far[next_tuple] = new_cost
                # Priority = cost + heuristic (Manhattan distance to goal).
                priority = new_cost + manhattan_distance(next_state, goal)
                heapq.heappush(frontier, (priority, next_tuple))
    
    # If the goal is not reachable, return the total number of actions taken.
    return -1

In [ ]:
cost = a_star(state, goal_state)
print(cost)

From experiments the best heuristic for this problem is the manhattan disatnce